# CPS4

## Brownian Generator code

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 27 22:49:33 2018

@author: yassine
"""

################################################
######## Importing useful modules ##############
################################################

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from numba import jit
import scipy.stats as sps

%matplotlib qt4


@jit
def Brownian_Generator(T,n,N):
    """
        This function generates N trajectories of the Brownian motion W with T/n the step of dicretization
        
        and [0,T] is the time interval on which we simulate.
        
        Each column of W is a trajectory among the N trajectories.
    
    """
    
    
    Delta_T = T/n
    Z = np.random.normal(0,1,size=(n,N))
    
    W = np.zeros((n+1,N))
    W[1:,:] = np.cumsum(np.sqrt(Delta_T)*Z,axis=0)
    
    return W


## Question 1

In [4]:
r = 0.03
sigma = 0.3
S0 = 100

mu = 0.05 
#mu = 0.02
#mu = 0.45

T = 1
n = 100
N = 1000



def S_generator(S0,T,n,N):

    times = (np.linspace(0,T,n+1)).reshape((n+1,1))

    W_copies = Brownian_Generator(T,n,N)
    S_copies = S0 * np.exp((mu- (sigma**2/2) )*times)  * np.exp(sigma*W_copies)

    return S_copies
    
    
times = (np.linspace(0,T,n+1)).reshape((n+1,1)) 

S_copies = S_generator(S0,T,n,N)
S_mean = np.mean(S_copies,axis=1)



plt.plot(times,S0*np.exp(mu*times),color = 'red',lw=1,label = "Exact mean")
plt.plot(times,S_mean,color = 'green',lw=0.7,label = "Monte-Carlo mean estimation")

plt.legend(loc="best")

## Question 2

In [12]:
def d_plus(s,k,v):
    return np.log(s/k)/np.sqrt(v) + np.sqrt(v)/2
    
def d_minus(s,k,v):
    return np.log(s/k)/np.sqrt(v) - np.sqrt(v)/2

def Delta(St,K,DT):    
    return sps.norm.cdf( d_plus(St , K*np.exp(-r*DT) , (sigma**2)*DT) )
    
def BS(S0,K,T):
    return S0*sps.norm.cdf( d_plus(S0, K*np.exp(-r*T), (sigma**2)*T) ) - K*np.exp(-r*T)*sps.norm.cdf( d_minus(S0, K*np.exp(-r*T), (sigma**2)*T) )

BS(S0,100,T)

    

13.283308397880909

## 2-a

In [14]:
r = 0.03
sigma = 0.3
S0 = 100

mu = 0.05 
#mu = 0.02
#mu = 0.45

T = 1
n = 100
N = 1000



def X_generator(K_list,S0,T,n,N):
    
    
    times = (np.linspace(0,T,n+1)).reshape((n+1,1))
    S_copies = S_generator(S0,T,n,N)
    
    X_copies = np.zeros((41,N))

    for i in range(41):

        K = K_list[i]

        delta_t_K = Delta( S_copies[:n,], K, T-times[:n] )
        actualisedS_copies = np.exp(-r*times)*S_copies

        X_copies[i,:] = BS(S0,100,T)+ np.sum(delta_t_K*(actualisedS_copies[1:,:] - actualisedS_copies[:n,:]),axis=0)

    return X_copies



def PL_Generator(K_list,S0,T,n,N):
    
    times = (np.linspace(0,T,n+1)).reshape((n+1,1))
    S_copies = S_generator(S0,T,n,N)
    
    X_copies = np.zeros((len(K_list),N))
    PayOff = np.zeros((len(K_list),N))
    
    for i in range(len(K_list)):

        K = K_list[i]

        delta_t_K = Delta( S_copies[:n,], K, T-times[:n] )
        actualisedS_copies = np.exp(-r*times)*S_copies

        X_copies[i,:] = BS(S0,100,T)+ np.sum(delta_t_K*(actualisedS_copies[1:,:] - actualisedS_copies[:n,:]),axis=0)

        PayOff[i,:] = (S_copies[-1,:] - K)*((S_copies[-1,:] - K)>0)
        
        
    PL_copies = X_copies - PayOff
    
    return PL_copies
    
    


In [15]:
r = 0.03
sigma = 0.3
S0 = 100

mu = 0.05 
#mu = 0.02
#mu = 0.45

T = 1
N = 10000



K_list = np.arange(80,121)
n_list = np.arange(100,120)


PL_mean_matrix = np.zeros((len(K_list),len(n_list)))
PL_std_matrix = np.zeros((len(K_list),len(n_list)))

for i in range(len(n_list)):
    
    n = n_list[i]
    PL = PL_Generator(K_list,S0,T,n,N)
    
    PL_mean_matrix[:,i] = np.mean(PL,axis=1)
    PL_std_matrix[:,i] = np.std(PL,axis=1)


In [16]:
X,Y = np.meshgrid(n_list,K_list)


fig1 = plt.figure(1)
ax1 = fig1.gca(projection='3d')
ax1.set_xlabel('n values')
ax1.set_ylabel('K values')
ax1.set_zlabel('mean of PL')

surf = ax1.plot_surface(X, Y, PL_mean_matrix,color = "orange")


fig2 = plt.figure(2)
ax2 = fig2.gca(projection='3d')
ax2.set_xlabel('n values')
ax2.set_ylabel('K values')
ax2.set_zlabel('mean of PL')

surf = ax2.plot_surface(X, Y, PL_std_matrix, color = "blue")



